# Combining original and flipped segmentations to get purely right-ventricle segmentations

In [1]:
from pathlib import Path
import sys

import cv2
import numpy as np
from tqdm import tqdm

import echonet

# print(cv2.getBuildInformation())

In [2]:
LEFT_SEGMENT_DIR = Path("output/segmentation/all-patients")
RIGHT_SEGMENT_DIR = Path("output/segmentation/flipped")
ECHONET_VIDEO_DIR = Path("/home/lex/data/echonet-data/Videos")
OUTPUT_DIR = Path("output/right-only-segmentation")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RED = np.array([255, 0, 0])
GREEN = np.array([0, 255, 0])
BLUE = np.array([0, 0, 255])
ORANGE = np.array([255, 165, 0])
LIGHT_GREY = np.array([211, 211, 211])

In [3]:
def get_heights(mask: np.ndarray):
    """Returns array of heights of mask for each frame!"""
    frame_indices, row_indices = np.where(mask.any(axis=1)==True)

    heights = []
    for frame_index in range(mask.shape[0]):
        this_frame = frame_indices == frame_index

        if len(row_indices[this_frame]) == 0:
            # print(f"Frame #{frame_index} appears to have no segmentations? Treating this as zero height...")
            heights.append(0)
            continue

        min_row, max_row = (min(row_indices[this_frame]), max(row_indices[this_frame]))
        height = max_row - min_row
        heights.append(height)

    heights = np.array(heights)
    return heights

In [ ]:
left_segment_mask_fps = [p for p in (LEFT_SEGMENT_DIR / "segmentation_masks").iterdir()]
for left_segment_mask_fp in tqdm(left_segment_mask_fps):
    # Get associated segment masks for this particular video
    # video_mask_name = video_fp.with_suffix(".npy").name # Turns "path/to/video.avi" into "video.npy"
    video_stem = Path(left_segment_mask_fp.stem) # turns "path/to/0x12345.npy" into just "0x12345"
    right_segment_mask_fp = RIGHT_SEGMENT_DIR / "segmentation_masks" / video_stem.with_suffix(".npy")
    echonet_video_fp = ECHONET_VIDEO_DIR / video_stem.with_suffix(".avi")
    # print(f"Processing {echonet_video_fp}...")

    left_segment_mask = np.load(left_segment_mask_fp)
    right_segment_mask = np.load(right_segment_mask_fp)
    echonet_video = echonet.utils.loadvideo(str(echonet_video_fp))

    # Since right mask comes from flipped version of video, we need to flip it back to normal (i.e. left-to-right)
    right_segment_mask = np.flip(right_segment_mask, axis=-1)

    # Keep track of the mistaken left ventricle segmentation by the "right" segmentation mask
    mistaken_left_mask = left_segment_mask & right_segment_mask
    # Subtract those mistaken left ventricle parts from the right segmentation
    right_only_mask = right_segment_mask ^ (mistaken_left_mask)

    # Get heights of left and right ventricles, check if right ventricle appears unreasonably large (i.e. probably including atrium accidentally!)
    left_heights = get_heights(left_segment_mask)
    right_heights = get_heights(right_only_mask)
    frames_with_too_large_RV_mask = right_heights > 0.8 * left_heights

    # Put colour channels at end to make it easier to assign pixel colours
    echonet_video = echonet_video.transpose((1, 2, 3, 0)) # I.e. now (frames, height, width, colours)
    echonet_video[left_segment_mask] = RED
    echonet_video[right_only_mask] = BLUE
    echonet_video[mistaken_left_mask] = GREEN
        
    border_thickness = 2
    echonet_video[frames_with_too_large_RV_mask, 0:border_thickness, :] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, -border_thickness:, :] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, :, 0:border_thickness] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, :, -border_thickness:] = ORANGE

    # Transpose video back to original shape now and save
    echonet_video = echonet_video.transpose((3, 0, 1, 2)) # now (colours, frames, height, width) as before
    output_fp = OUTPUT_DIR / video_stem.with_suffix(".avi")

    echonet.utils.savevideo(str(output_fp), echonet_video, 30)

# Filtering out garbage data
Here, we'll be trying to discard any videos that are not A4C views.

In [19]:
VIDEONAME = "0X1A8D85542DBE8204"

LEFT_SEGMENTATION_MASK_FP = Path(f"/home/lex/Development/ultrasound/output/segmentation/all-patients/segmentation_masks/{VIDEONAME}.npy")
RIGHT_SEGMENTATION_MASK_FP = Path(f"/home/lex/Development/ultrasound/output/segmentation/flipped/segmentation_masks/{VIDEONAME}.npy")
ECHONET_VIDEO_FP = Path(f"/home/lex/data/echonet-data/Videos/{VIDEONAME}.avi")

left_segmentation_mask = np.load(LEFT_SEGMENTATION_MASK_FP)
right_segmentation_mask = np.load(RIGHT_SEGMENTATION_MASK_FP)
intersection_mask = left_segmentation_mask & right_segmentation_mask
# Subtract those mistaken left ventricle parts from the right segmentation
right_only_mask = right_segmentation_mask ^ intersection_mask


echonet_video = echonet.utils.loadvideo(str(ECHONET_VIDEO_FP))

num_frames, height, width = right_segmentation_mask.shape

WINDOW = "Mask"
# cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

i = 0
is_playing = True
was_seeking = False
# while True:
#     if i >= num_frames:
#         i = 0
#         print("Video looped!")
#     elif i < 0:
#         i = num_frames - 1

#     right_mask = right_segmentation_mask[i].copy()
#     left_mask = left_segmentation_mask[i].copy()
#     and_mask = intersection_mask[i].copy()

#     mistaken_left_mask = left_mask & right_mask
#     # Subtract those mistaken left ventricle parts from the right segmentation
#     right_only_mask = right_mask ^ (mistaken_left_mask)

#     right_area = right_mask.sum()
#     left_area = left_mask.sum()
#     and_area = and_mask.sum()
    
#     print(f"and/left={and_area/left_area*100:.2f};    and/right={}")
#     # print(f"right: {right_mask.sum()}; left: {left_mask.sum()}; intersection: {and_mask.sum()}")





#     # right_mask = right_mask.astype(np.uint8) * 255
    
#     # contours, hierarchy = cv2.findContours(right_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     # areas = [cv2.contourArea(contour) for contour in contours]
    
#     # max_area = max(areas)
#     # max_index = areas.index(max_area)
#     # max_contour = contours[max_index]

#     # right_mask = cv2.cvtColor(right_mask, cv2.COLOR_GRAY2RGB)
#     # cv2.drawContours(right_mask, [max_contour], -1, (0,255,0), 1)

#     # height, width, channels = right_mask.shape
#     # top_border = np.zeros((height // 8, width, channels), dtype=right_mask.dtype)
#     # top_border[:, :] = np.expand_dims(LIGHT_GREY, (0, 1))
#     # cv2.putText(top_border, f"Frame {i+1}/{num_frames}", org=(5,10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=(255, 0, 0))
#     # right_mask = np.concatenate([top_border, right_mask], axis=0)
#     # cv2.imshow("Mask", right_mask)

#     keypress = cv2.waitKey(50) & 0xFF
#     if keypress == ord('q'):
#         break
#     elif keypress == ord(' '):
#         is_playing = not is_playing
#     elif keypress == ord('a'):
#         i -= 1
#     elif keypress == ord('d'):
#         i += 1
#     else:
#         if is_playing:
#             i += 1

# cv2.destroyAllWindows()

In [ ]:
left_segmentation_mask.shape

In [ ]:
echonet_video.shape

In [20]:
right_only_mask.sum(axis=(1, 2, 0)) / intersection_mask.sum(axis=(1, 2, 0))

1.278674057527371